# 1. Test Random Environment with OpenAI GYM

In [1]:
import gym 
import random

In [3]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
states, actions

(4, 2)

In [6]:
episodes = 10
for e in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0, 1])
        next_step, reward, done, info =  env.step(action)
        score += reward
    
    print(f"Episode: {e}, Score: {score}")

Episode: 1, Score: 14.0
Episode: 2, Score: 22.0
Episode: 3, Score: 33.0
Episode: 4, Score: 37.0
Episode: 5, Score: 14.0
Episode: 6, Score: 43.0
Episode: 7, Score: 13.0
Episode: 8, Score: 26.0
Episode: 9, Score: 45.0
Episode: 10, Score: 23.0


# 2. Create a Deep Learning Model with Keras

In [19]:
import numpy as np 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24,  activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
model = build_model(states, actions)

In [ ]:
del model

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [12]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(
        model=model,
        memory=memory,
        policy=policy,
        nb_actions=actions, 
        nb_steps_warmup=10,
        target_model_update=1e-2
    )
    return dqn

In [20]:
dqn = build_agent(model, actions)
dqn.compile(
    optimizer=Adam(learning_rate=1e-3),
    metrics=['mae']
)

In [21]:
dqn.fit(
    env, nb_steps=50000,
    visualize=False,
    verbose=1
)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 22:05 - reward: 1.0000

c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 150s 15ms/step - reward: 1.0000
101 episodes - episode_reward: 98.208 [10.000, 442.000] - loss: 2.017 - mae: 19.301 - mean_q: 39.119

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 150s 15ms/step - reward: 1.0000
46 episodes - episode_reward: 214.848 [158.000, 318.000] - loss: 2.496 - mae: 40.726 - mean_q: 82.340

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 115s 12ms/step - reward: 1.0000
44 episodes - episode_reward: 229.000 [169.000, 332.000] - loss: 2.221 - mae: 44.576 - mean_q: 90.036

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 147s 15ms/step - reward: 1.0000
44 episodes - episode_reward: 227.023 [131.000, 500.000] - loss: 2.398 - mae: 46.192 - mean_q: 93.134

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: 1.0000
done, took 651.755 seconds


In [22]:
scores = dqn.test(env, nb_episodes=100, visualize=False)

Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 482.000, steps: 482
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 

KeyError: 'episod_reward'

In [24]:
print(np.mean(scores.history['episode_reward']))

498.74


In [25]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500


# 4. Reload Agent from Memory

In [28]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [29]:
del env
del model
del dqn

In [33]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [34]:
dqn.load_weights('dqn_weights.h5f')

In [35]:
dqn.test(env, nb_episodes=3, visualize=True)

Testing for 3 episodes ...


c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
Exception ignored in: <function Viewer.__del__ at 0x0000025C288D6EF0>
Traceback (most recent call last):
  File "c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site-packages\pyglet\window\win32\__init__.py", line 299, in close
    super(Win32Window, self).close()
  File "c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\ml\lib\site

Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500


In [36]:
env.close()